In [ ]:
import msprime
import numpy as np
import pandas as pd
import math
from random import shuffle
from primeval import *

### Code for the Fu et al. (2012) model

### Actual function is found in primeval.py file

```python
def fu_model(mu=1.5e-8, phi=0, length=1e4, n_afr=0, n_eur=0, debug=False):

    generation_time = 25
    
    # 220kya:
    # African population constant with Ne~7300
    N_A = 7310
    
    # 148kya:
    # instantaneous growth to Ne~14000
    T_AF = 148e3 / generation_time
    N_AF = 14474
    
    # 51kya:
    # non-AFR pops migrate OOA; bottlenecks to Ne~1800
    # migration between AFR occurs
    N_B = 1861
    T_SPLIT = 51e3 / generation_time
    m_AF_B = 15e-5
    
    # 23kya:
    # 2nd EUR bottlenecks to Ne~1000 & starts growing with rate 0.307%
    # migration rate slows between AFR-EUR
    N_EU0 = 1032
    T_EU_B = 23e3 / generation_time
    m_AF_EU = 2.5e-5
    r_EU0 = 0.00307
    N_EU1 = N_EU0 / math.exp(-r_EU0 * T_EU_B)
    
    # 5.1kya:
    # explosive growth in both AFR & EUR
    T_EG = 5.1e3 / generation_time
    r_EU = 0.0195
    r_AF = 0.0166
    N_EU_start = N_EU1 / math.exp(-r_EU * T_EG)
    m_EG = 0
    N_AF_start = N_AF / math.exp(-r_AF * T_EG)
    
    # Population IDs correspond to their indexes in the population
    # configuration array. Therefore, we have 0=YRI, 1=CEU initially.
    population_configurations = [
        msprime.PopulationConfiguration(
            sample_size=n_afr, initial_size=N_AF_start, growth_rate=r_AF),
        msprime.PopulationConfiguration(
            sample_size=n_eur, initial_size=N_EU_start, growth_rate=r_EU)#,
    ]

    # up to 5.1kya, no migration
    migration_matrix = [
        [0, 0],
        [0, 0],
    ]
    
    demographic_events = [
        # at 5.1kya, change to slow growth rate in EUR & stop growth in AFR;
        # add migration rate
        msprime.MigrationRateChange(
            time=T_EG, rate=m_AF_EU, matrix_index=(0, 1)),
        msprime.MigrationRateChange(
            time=T_EG, rate=m_AF_EU, matrix_index=(1, 0)),
        msprime.PopulationParametersChange(
            time=T_EG, growth_rate=r_EU0, initial_size=N_EU1, population_id=1),
        msprime.PopulationParametersChange(
            time=T_EG, growth_rate=0, population_id=0),
        
        # at 23kya, EUR growth stops and migration rates increase
        msprime.MigrationRateChange(
            time=T_EU_B, rate=m_AF_B, matrix_index=(0, 1)),
        msprime.MigrationRateChange(
            time=T_EU_B, rate=m_AF_B, matrix_index=(1, 0)),
        msprime.PopulationParametersChange(
            time=T_EU_B, initial_size=N_EU0, growth_rate=0, population_id=1),
        
        # at 51kya, population B merges into AFR
        msprime.MassMigration(
            time=T_SPLIT, source=1, destination=0, proportion=1.0),
        msprime.PopulationParametersChange(
            time=T_SPLIT, initial_size=N_B, population_id=1),
        
        # At 148kya, instantaneous growth in AFR
        msprime.PopulationParametersChange(
            time=T_AF, initial_size=N_A, population_id=0)
    ]
    
    if(debug):
        # Use the demography debugger to print out the demographic history
        # that we have just described.
        dd = msprime.DemographyDebugger(
            population_configurations=population_configurations,
            migration_matrix=migration_matrix,
            demographic_events=demographic_events)
        dd.print_history()
    else:
        sim = msprime.simulate(population_configurations=population_configurations,
                               migration_matrix=migration_matrix, 
                               demographic_events=demographic_events,
                               mutation_rate=mu, 
                               recombination_rate=phi, 
                               length=length,
                               random_seed=30)
        return sim
```

### Run with debugging to check correct parameters at each epoch

In [ ]:
fu_model(debug=True)
# fu_model(mu=1.5e-8, phi=2e-8, length=5e7, n_afr=0, n_eur=2000, debug=True)